In [1]:
from all_func import *
from langchain_ollama import OllamaLLM, ChatOllama
from pprint import pprint

In [2]:
llm = ChatOllama(model="llama3.1", temperature=0.5)

In [ ]:
tools = [power, get_weather_info, calculate_trip_cost, calculate_bmi, get_capital, count_letter, calculate_area, get_timezone, zodiac_sign, date_difference, calculate_tip, playlist_duration_calculator, sentiment_analysis, simple_interest, password_generator]

llm_with_tools = llm.bind_tools(tools)

In [4]:
pprint(get_timezone.args)
pprint(count_letter.args)

{'city': {'title': 'City', 'type': 'string'},
 'country': {'title': 'Country', 'type': 'string'},
 'mail_code': {'title': 'Mail Code', 'type': 'string'},
 'state': {'title': 'State', 'type': 'string'}}
{'letter': {'title': 'Letter', 'type': 'string'},
 'word': {'title': 'Word', 'type': 'string'}}


In [5]:
get_timezone.invoke({"city": "Ketchikan", "state": "Alaska", "country": "USA", "mail_code": "99950"})

'{"result": "America/Sitka"}'

In [6]:
msg = llm_with_tools.invoke("24 raised to 3")

In [7]:
msg.tool_calls

[{'name': 'difference_of_squares',
  'args': {},
  'id': 'bbe002f8-245b-416f-b4f6-b8ff9d81d079',
  'type': 'tool_call'}]

In [8]:
from operator import itemgetter

chain = llm_with_tools | (lambda x: x.tool_calls[0]["args"]) | power
chain.invoke("124 to the power of 3")

ValidationError: 2 validation errors for power
base
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing
exponent
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing

In [ ]:
from tqdm import tqdm
from time import sleep

chain = llm_with_tools | (lambda x: x.tool_calls[0]["args"]) | get_timezone
results = []
for i in tqdm(range(100)):
    sleep(15)
    try:
        result = chain.invoke("What is the timezone of Ketchikan, Alaska, USA 99950?")
        results.append(result)
    except Exception as e:
        results.append("failed")

100%|██████████| 100/100 [29:15<00:00, 17.55s/it]


In [ ]:
failed_count = results.count("failed")
print(failed_count)
correct_count = results.count('{"result": "America/Sitka"}')
print(correct_count)

0
100
